<div id="container" style="position:relative;">
<div style="float:left"><h1> Forecasting Bakery Sales - Abi Magnall </h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://twomagpiesbakery.co.uk/wp-content/uploads/2020/11/logo-no-site.jpg" />
</div>
</div>

# Notebook 1 : Bakery Data Gathering

---

The purpose of this notebook is to explain the problem and purpose of this project, as well as collect the data from the a cloud based system using their API. 

A total of four bakeries, which will be referred to as `Aldeburgh`, `Southwold`, `Darsham` and `Norwich` throughout this project, will have the transaction data collected spanning from 01-09-2020 to 31-10-2022. This data was provided by the amazing `Two Magpies Bakery` in order to improve their business planning and strategy using the outcomes of this project. 

**N.B** This notebook does not need to be run, it only contains cells to make the API calls to the till system, and therefore will not run without the API tokens. 

---

# Contents  

**1. [Introduction](#Introduction)**
- [Problem Statement](#Problem-Statement)


**2. [Data Collection](#Data-Collection)**
- [Data and Column Descriptions](#Data-and-Column-Descriptions)

**3. [Next Steps](#Next-Steps)**

___

# Introduction 
## Problem Statement 

For any business cash flow is one of the most important factors that determines whether a business will not only grow but even survive. It is essential for a business to know how much revenue they're going to generate and receive throughout the year so that they can plan their expansion strategies appropriately and have security in knowledge that they can operate effectively. Without having an accurate data driven forecast, all the planning and strategy would be guesswork. Therefore, it is essential for the success and growth of any business to have a reliable forecast.



    "The two most important things about forecast accuracy are:
    1. Surety of cash and so giving confidence to the banks and any potential investors
    2. To manage & optimise manufacturing efficiency - ensuring that there is the right labour producing the right product at the right time" - Co-Owner, Two Magpies Bakery



**The Value Add of an Accurate Data-Driven Revenue Forecast:**
- Managers can be proactive instead of reactive and can adapt their strategy to achieve the sales needed
- Better investments, staffing and hiring decisions based on peaks and troughs throughout the year (achieved through a weekly and monthly forecast) 
- A monthly forecast is required to show to potential investors to raise funding as it shows that the business if not only target but going to grow

All of these are required to support the growth of the business. 

**The Aim of this Project:**

The aim of this project is to develop accurate revenue forecasts for a leading Artisan Bakery in East Anglia, called the [Two Magpies Bakery](https://twomagpiesbakery.co.uk/). The bakery requires: 
1. A daily forecast up to 7 days ahead, *with a target accuracy of 95%*
2. A weekly forecast up to 6 weeks ahead, *with a target accuracy of 95%*
3. A monthly forecast up to 6 months ahead, *with a target accuracy of 92-95%* 

---

# Data Collection
The data was collected calling an API to the EposNow till system used by the bakery. The API tokens are saved in external text files which are called in, therefore the below code blocks performing the API call will not run without the tokens. 

The API gets the data between two given dates, which gets appended to a dataframe. The newly formed dataframe has rows of data, where each row of data is a new transaction. 

The full raw dataset runs from 01/09/2020-31/10/2022. These dates were selected as it the longest amount of time that was available post the main lockdowns due to the Corona virus pandemic. This size of data is the minimum required for timeseries analysis, and potentially may prove problematic due to the length. However, speaking with the bakery owner, the sales and spending habits of customers have completely changed since the pandemic. Therefore, it was thought that collecting data pre-pandemic would have different revenue trends that would be identified and forecasted by the models, which are not longer accurate for the true patterns. 

## Imports

In [49]:
import requests
import pandas as pd
import os

## To Get Current Directory

In [48]:
working_directory = os.getcwd()
working_directory

'/Users/abimagnall/Documents/BrainStation/Capstone/Data'

## Importing Custom Functions

In [2]:
import BakeryFunctions as bakery

## Importing the API Tokens

In [13]:
ald_token = open('aldeburgh_token.txt', 'r')
sw_token = open('southwold_token.txt', 'r')
dar_token = open('darsham_token.txt', 'r')
nor_token = open('norwich_token.txt', 'r')

---

# Aldeburgh Data Download 
*The inputted data is for the final download of data which was for the test set spanning 01-10-2022 - 01-11-2022, not the full data set from 01-09-2020.*

The raw data is returned as a list of dictionaries, where each dictionary contains the transaction information. This is identified and retrieved from the full list and gets appended to the dataframe. 

In [19]:
# Aldeburgh Download

# An empty dataframe is created to be filled with transaction data 
aldeburgh_df = pd.DataFrame()

# To loop through the page numbers 
for i in range(1, 65): 
    # Link to call the API which gets the transaction rows between two dates 
    api_url = 'https://api.eposnowhq.com/api/v4/Transaction/GetByDate/2022-10-01/2022-11-01/?page='+str(i)
    # To call the API token 
    token = ald_token.read()
    headers = {'Authorization': "Basic {} ".format(token)}
    auth_response = requests.get(api_url, headers=headers)
    print(f'Processing page {i}: {len(auth_response.json())} transactions found.')
    for j in range(200):
        try:
            # To append the data in the correct format, with the correct date for each transaction 
            for transaction_item in auth_response.json()[j]['TransactionItems']:
                tmp = pd.DataFrame.from_dict(transaction_item, orient = 'index').T
                tmp['Date'] = auth_response.json()[j]['DateTime']
                #print(tmp)
                try:
                    aldeburgh_df = pd.concat([aldeburgh_df, tmp])
                except:
                    # To print if there are any issues processing that transaction row 
                    print(f'Issue processing transaction: page {i}, transaction {j}')
        except:
            print(f'Issue processing transaction: page {i}, transaction {j}')

Processing page 1: 200 transactions found.
Issue processing transaction: page 1, transaction 12


In [20]:
# To validate it downloaded successfully 
aldeburgh_df.head()

,Id,TransactionId,ProductId,UnitPrice,UnitPriceExcTax,CostPrice,TaxGroupId,TaxAmount,Quantity,DiscountAmount,...,Notes,PrintOnOrder,MultipleChoiceProductId,ParentId,IsTaxExempt,MeasurementDetails,Taxes,MultipleChoiceItems,CourseFired,Date
0,1639859367,467709962,1677556,3.1,3.1,0.56,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859367, 'TaxRateId'...",[],False,2022-10-01T08:02:15.543
0,1639859369,467709962,1677551,2.5,2.5,0.25,None,0.0,2,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859369, 'TaxRateId'...",[],False,2022-10-01T08:02:15.543
0,1639859371,467709962,15648142,2.75,2.75,0.0,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859371, 'TaxRateId'...",[],False,2022-10-01T08:02:15.543
0,1639859629,467710056,1677390,3.6,3.0,0.34,None,0.6,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859629, 'TaxRateId'...","[{'Id': 1639859634, 'TransactionId': 467710056...",False,2022-10-01T08:04:42.373
0,1639859631,467710056,1677390,3.6,3.0,0.34,None,0.6,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859631, 'TaxRateId'...",[],False,2022-10-01T08:04:42.373


In [8]:
# To export the the data to a new csv file 
aldeburgh_df.to_csv(working_directory+'/1_raw_data/ald_oct22.csv',index=False)

## Observations 
Whilst the data was being called from the API, some transactions had issues processing them. This will therefore be explored in [Bakery Data Preprocessing](./4_Bakery_Data_Preprocessing.ipynb) to ensure that all transaction rows are accounted for but comapring the calculated revenue to the true revenue. 

---

# Southwold Data Download

In [9]:
# Southwold Download

# A new empty dataframe is created 
southwold_df = pd.DataFrame()
# To loop through the page numbers 
for i in range(1, 65): 
    # To call the API 
    api_url = 'https://api.eposnowhq.com/api/v4/Transaction/GetByDate/2022-10-01/2022-11-01/?page='+str(i)
    token = sw_token.read()
    headers = {'Authorization': "Basic {} ".format(token)}
    auth_response = requests.get(api_url, headers=headers)
    print(f'Processing page {i}: {len(auth_response.json())} transactions found.')
    for j in range(200):
        try:
            for transaction_item in auth_response.json()[j]['TransactionItems']:
                tmp = pd.DataFrame.from_dict(transaction_item, orient = 'index').T
                tmp['Date'] = auth_response.json()[j]['DateTime']
                try:
                    southwold_df = pd.concat([southwold_df, tmp])
                except:
                    print(f'Issue processing transaction: page {i}, transaction {j}')
        except:
            print(f'Issue processing transaction: page {i}, transaction {j}')

Processing page 1: 200 transactions found.
Processing page 2: 200 transactions found.
Processing page 3: 200 transactions found.
Processing page 4: 200 transactions found.
Processing page 5: 200 transactions found.
Processing page 6: 200 transactions found.
Processing page 7: 200 transactions found.
Processing page 8: 200 transactions found.
Processing page 9: 200 transactions found.
Processing page 10: 200 transactions found.
Processing page 11: 200 transactions found.
Processing page 12: 200 transactions found.
Processing page 13: 200 transactions found.
Processing page 14: 200 transactions found.
Processing page 15: 200 transactions found.
Processing page 16: 200 transactions found.
Processing page 17: 200 transactions found.
Processing page 18: 200 transactions found.
Processing page 19: 200 transactions found.
Processing page 20: 200 transactions found.
Processing page 21: 200 transactions found.
Processing page 22: 200 transactions found.
Processing page 23: 200 transactions foun

Processing page 49: 0 transactions found.
Issue processing transaction: page 49, transaction 0
Issue processing transaction: page 49, transaction 1
Issue processing transaction: page 49, transaction 2
Issue processing transaction: page 49, transaction 3
Issue processing transaction: page 49, transaction 4
Issue processing transaction: page 49, transaction 5
Issue processing transaction: page 49, transaction 6
Issue processing transaction: page 49, transaction 7
Issue processing transaction: page 49, transaction 8
Issue processing transaction: page 49, transaction 9
Issue processing transaction: page 49, transaction 10
Issue processing transaction: page 49, transaction 11
Issue processing transaction: page 49, transaction 12
Issue processing transaction: page 49, transaction 13
Issue processing transaction: page 49, transaction 14
Issue processing transaction: page 49, transaction 15
Issue processing transaction: page 49, transaction 16
Issue processing transaction: page 49, transaction

Processing page 50: 0 transactions found.
Issue processing transaction: page 50, transaction 0
Issue processing transaction: page 50, transaction 1
Issue processing transaction: page 50, transaction 2
Issue processing transaction: page 50, transaction 3
Issue processing transaction: page 50, transaction 4
Issue processing transaction: page 50, transaction 5
Issue processing transaction: page 50, transaction 6
Issue processing transaction: page 50, transaction 7
Issue processing transaction: page 50, transaction 8
Issue processing transaction: page 50, transaction 9
Issue processing transaction: page 50, transaction 10
Issue processing transaction: page 50, transaction 11
Issue processing transaction: page 50, transaction 12
Issue processing transaction: page 50, transaction 13
Issue processing transaction: page 50, transaction 14
Issue processing transaction: page 50, transaction 15
Issue processing transaction: page 50, transaction 16
Issue processing transaction: page 50, transaction

Processing page 51: 0 transactions found.
Issue processing transaction: page 51, transaction 0
Issue processing transaction: page 51, transaction 1
Issue processing transaction: page 51, transaction 2
Issue processing transaction: page 51, transaction 3
Issue processing transaction: page 51, transaction 4
Issue processing transaction: page 51, transaction 5
Issue processing transaction: page 51, transaction 6
Issue processing transaction: page 51, transaction 7
Issue processing transaction: page 51, transaction 8
Issue processing transaction: page 51, transaction 9
Issue processing transaction: page 51, transaction 10
Issue processing transaction: page 51, transaction 11
Issue processing transaction: page 51, transaction 12
Issue processing transaction: page 51, transaction 13
Issue processing transaction: page 51, transaction 14
Issue processing transaction: page 51, transaction 15
Issue processing transaction: page 51, transaction 16
Issue processing transaction: page 51, transaction

Processing page 52: 0 transactions found.
Issue processing transaction: page 52, transaction 0
Issue processing transaction: page 52, transaction 1
Issue processing transaction: page 52, transaction 2
Issue processing transaction: page 52, transaction 3
Issue processing transaction: page 52, transaction 4
Issue processing transaction: page 52, transaction 5
Issue processing transaction: page 52, transaction 6
Issue processing transaction: page 52, transaction 7
Issue processing transaction: page 52, transaction 8
Issue processing transaction: page 52, transaction 9
Issue processing transaction: page 52, transaction 10
Issue processing transaction: page 52, transaction 11
Issue processing transaction: page 52, transaction 12
Issue processing transaction: page 52, transaction 13
Issue processing transaction: page 52, transaction 14
Issue processing transaction: page 52, transaction 15
Issue processing transaction: page 52, transaction 16
Issue processing transaction: page 52, transaction

...

Processing page 60: 0 transactions found.
Issue processing transaction: page 60, transaction 0
Issue processing transaction: page 60, transaction 1
Issue processing transaction: page 60, transaction 2
Issue processing transaction: page 60, transaction 3
Issue processing transaction: page 60, transaction 4
Issue processing transaction: page 60, transaction 5
Issue processing transaction: page 60, transaction 6
Issue processing transaction: page 60, transaction 7
Issue processing transaction: page 60, transaction 8
Issue processing transaction: page 60, transaction 9
Issue processing transaction: page 60, transaction 10
Issue processing transaction: page 60, transaction 11
Issue processing transaction: page 60, transaction 12
Issue processing transaction: page 60, transaction 13
Issue processing transaction: page 60, transaction 14
Issue processing transaction: page 60, transaction 15
Issue processing transaction: page 60, transaction 16
Issue processing transaction: page 60, transaction

Processing page 61: 0 transactions found.
Issue processing transaction: page 61, transaction 0
Issue processing transaction: page 61, transaction 1
Issue processing transaction: page 61, transaction 2
Issue processing transaction: page 61, transaction 3
Issue processing transaction: page 61, transaction 4
Issue processing transaction: page 61, transaction 5
Issue processing transaction: page 61, transaction 6
Issue processing transaction: page 61, transaction 7
Issue processing transaction: page 61, transaction 8
Issue processing transaction: page 61, transaction 9
Issue processing transaction: page 61, transaction 10
Issue processing transaction: page 61, transaction 11
Issue processing transaction: page 61, transaction 12
Issue processing transaction: page 61, transaction 13
Issue processing transaction: page 61, transaction 14
Issue processing transaction: page 61, transaction 15
Issue processing transaction: page 61, transaction 16
Issue processing transaction: page 61, transaction

Processing page 62: 0 transactions found.
Issue processing transaction: page 62, transaction 0
Issue processing transaction: page 62, transaction 1
Issue processing transaction: page 62, transaction 2
Issue processing transaction: page 62, transaction 3
Issue processing transaction: page 62, transaction 4
Issue processing transaction: page 62, transaction 5
Issue processing transaction: page 62, transaction 6
Issue processing transaction: page 62, transaction 7
Issue processing transaction: page 62, transaction 8
Issue processing transaction: page 62, transaction 9
Issue processing transaction: page 62, transaction 10
Issue processing transaction: page 62, transaction 11
Issue processing transaction: page 62, transaction 12
Issue processing transaction: page 62, transaction 13
Issue processing transaction: page 62, transaction 14
Issue processing transaction: page 62, transaction 15
Issue processing transaction: page 62, transaction 16
Issue processing transaction: page 62, transaction

Processing page 63: 0 transactions found.
Issue processing transaction: page 63, transaction 0
Issue processing transaction: page 63, transaction 1
Issue processing transaction: page 63, transaction 2
Issue processing transaction: page 63, transaction 3
Issue processing transaction: page 63, transaction 4
Issue processing transaction: page 63, transaction 5
Issue processing transaction: page 63, transaction 6
Issue processing transaction: page 63, transaction 7
Issue processing transaction: page 63, transaction 8
Issue processing transaction: page 63, transaction 9
Issue processing transaction: page 63, transaction 10
Issue processing transaction: page 63, transaction 11
Issue processing transaction: page 63, transaction 12
Issue processing transaction: page 63, transaction 13
Issue processing transaction: page 63, transaction 14
Issue processing transaction: page 63, transaction 15
Issue processing transaction: page 63, transaction 16
Issue processing transaction: page 63, transaction

Processing page 64: 0 transactions found.
Issue processing transaction: page 64, transaction 0
Issue processing transaction: page 64, transaction 1
Issue processing transaction: page 64, transaction 2
Issue processing transaction: page 64, transaction 3
Issue processing transaction: page 64, transaction 4
Issue processing transaction: page 64, transaction 5
Issue processing transaction: page 64, transaction 6
Issue processing transaction: page 64, transaction 7
Issue processing transaction: page 64, transaction 8
Issue processing transaction: page 64, transaction 9
Issue processing transaction: page 64, transaction 10
Issue processing transaction: page 64, transaction 11
Issue processing transaction: page 64, transaction 12
Issue processing transaction: page 64, transaction 13
Issue processing transaction: page 64, transaction 14
Issue processing transaction: page 64, transaction 15
Issue processing transaction: page 64, transaction 16
Issue processing transaction: page 64, transaction

In [12]:
# To validate it worked 
southwold_df

,Id,TransactionId,ProductId,UnitPrice,UnitPriceExcTax,CostPrice,TaxGroupId,TaxAmount,Quantity,DiscountAmount,...,Notes,PrintOnOrder,MultipleChoiceProductId,ParentId,IsTaxExempt,MeasurementDetails,Taxes,MultipleChoiceItems,CourseFired,Date
0,1639859999,467710192,7243504,2.5,2.5,0.21,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859999, 'TaxRateId'...",[],False,2022-10-01T08:07:42.647
0,1639859998,467710192,5453564,2.7,2.7,0.14,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859998, 'TaxRateId'...",[],False,2022-10-01T08:07:42.647
0,1639859997,467710192,13405926,3.0,3.0,0.0,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859997, 'TaxRateId'...",[],False,2022-10-01T08:07:42.647
0,1639859996,467710192,1677426,3.25,2.70833,0.34,None,0.54,1,None,...,Y,True,None,None,False,None,"[{'TransactionItemId': 1639859996, 'TaxRateId'...",[],False,2022-10-01T08:07:42.647
0,1639859995,467710192,1677429,2.8,2.33333,0.36,None,0.47,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859995, 'TaxRateId'...","[{'Id': 1639860000, 'TransactionId': 467710192...",False,2022-10-01T08:07:42.647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1662749758,473959802,1677539,2.3,2.3,0.28,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662749758, 'TaxRateId'...",[],False,2022-10-31T16:37:29.163
0,1662749757,473959802,5453564,2.7,2.7,0.14,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662749757, 'TaxRateId'...",[],False,2022-10-31T16:37:29.163
0,1662749756,473959802,5453564,2.7,2.7,0.14,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662749756, 'TaxRateId'...",[],False,2022-10-31T16:37:29.163
0,1662759190,473962862,1677568,2.75,2.75,0.35,None,0.0,2,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662759190, 'TaxRateId'...",[],False,2022-10-31T16:56:18.903


In [13]:
# To save the raw data to a csv file 
southwold_df.to_csv(working_directory+'/1_raw_data/sw_oct22.csv',index=False)

---

# Darsham Data Download

In [31]:
# Darsham Download

# Empty dataframe is created
darsham_df = pd.DataFrame()
# To loop through the page numbers 
for i in range(1, 65): 
    api_url = 'https://api.eposnowhq.com/api/v4/Transaction/GetByDate/2022-10-01/2022-11-01/?page='+str(i)
    token = dar_token.read()
    headers = {'Authorization': "Basic {} ".format(token)}
    auth_response = requests.get(api_url, headers=headers)
    print(f'Processing page {i}: {len(auth_response.json())} transactions found.')
    for j in range(200):
        try:
            for transaction_item in auth_response.json()[j]['TransactionItems']:
                tmp = pd.DataFrame.from_dict(transaction_item, orient = 'index').T
                tmp['Date'] = auth_response.json()[j]['DateTime']
                #print(tmp)
                try:
                    darsham_df = pd.concat([darsham_df, tmp])
                except:
                    print(f'Issue processing transaction: page {i}, transaction {j}')
        except:
            print(f'Issue processing transaction: page {i}, transaction {j}')

Processing page 1: 200 transactions found.
Processing page 2: 200 transactions found.
Processing page 3: 200 transactions found.
Issue processing transaction: page 3, transaction 119
Processing page 4: 200 transactions found.
Issue processing transaction: page 4, transaction 149
Processing page 5: 200 transactions found.
Processing page 6: 200 transactions found.
Processing page 7: 200 transactions found.
Processing page 8: 200 transactions found.
Processing page 9: 200 transactions found.
Processing page 10: 200 transactions found.
Processing page 11: 200 transactions found.
Processing page 12: 200 transactions found.
Processing page 13: 200 transactions found.
Processing page 14: 200 transactions found.
Processing page 15: 200 transactions found.
Processing page 16: 200 transactions found.
Issue processing transaction: page 16, transaction 112
Processing page 17: 200 transactions found.
Processing page 18: 200 transactions found.
Processing page 19: 200 transactions found.
Processing

Processing page 32: 0 transactions found.
Issue processing transaction: page 32, transaction 0
Issue processing transaction: page 32, transaction 1
Issue processing transaction: page 32, transaction 2
Issue processing transaction: page 32, transaction 3
Issue processing transaction: page 32, transaction 4
Issue processing transaction: page 32, transaction 5
Issue processing transaction: page 32, transaction 6
Issue processing transaction: page 32, transaction 7
Issue processing transaction: page 32, transaction 8
Issue processing transaction: page 32, transaction 9
Issue processing transaction: page 32, transaction 10
Issue processing transaction: page 32, transaction 11
Issue processing transaction: page 32, transaction 12
Issue processing transaction: page 32, transaction 13
Issue processing transaction: page 32, transaction 14
Issue processing transaction: page 32, transaction 15
Issue processing transaction: page 32, transaction 16
Issue processing transaction: page 32, transaction

Processing page 33: 0 transactions found.
Issue processing transaction: page 33, transaction 0
Issue processing transaction: page 33, transaction 1
Issue processing transaction: page 33, transaction 2
Issue processing transaction: page 33, transaction 3
Issue processing transaction: page 33, transaction 4
Issue processing transaction: page 33, transaction 5
Issue processing transaction: page 33, transaction 6
Issue processing transaction: page 33, transaction 7
Issue processing transaction: page 33, transaction 8
Issue processing transaction: page 33, transaction 9
Issue processing transaction: page 33, transaction 10
Issue processing transaction: page 33, transaction 11
Issue processing transaction: page 33, transaction 12
Issue processing transaction: page 33, transaction 13
Issue processing transaction: page 33, transaction 14
Issue processing transaction: page 33, transaction 15
Issue processing transaction: page 33, transaction 16
Issue processing transaction: page 33, transaction

Processing page 34: 0 transactions found.
Issue processing transaction: page 34, transaction 0
Issue processing transaction: page 34, transaction 1
Issue processing transaction: page 34, transaction 2
Issue processing transaction: page 34, transaction 3
Issue processing transaction: page 34, transaction 4
Issue processing transaction: page 34, transaction 5
Issue processing transaction: page 34, transaction 6
Issue processing transaction: page 34, transaction 7
Issue processing transaction: page 34, transaction 8
Issue processing transaction: page 34, transaction 9
Issue processing transaction: page 34, transaction 10
Issue processing transaction: page 34, transaction 11
Issue processing transaction: page 34, transaction 12
Issue processing transaction: page 34, transaction 13
Issue processing transaction: page 34, transaction 14
Issue processing transaction: page 34, transaction 15
Issue processing transaction: page 34, transaction 16
Issue processing transaction: page 34, transaction

Processing page 35: 0 transactions found.
Issue processing transaction: page 35, transaction 0
Issue processing transaction: page 35, transaction 1
Issue processing transaction: page 35, transaction 2
Issue processing transaction: page 35, transaction 3
Issue processing transaction: page 35, transaction 4
Issue processing transaction: page 35, transaction 5
Issue processing transaction: page 35, transaction 6
Issue processing transaction: page 35, transaction 7
Issue processing transaction: page 35, transaction 8
Issue processing transaction: page 35, transaction 9
Issue processing transaction: page 35, transaction 10
Issue processing transaction: page 35, transaction 11
Issue processing transaction: page 35, transaction 12
Issue processing transaction: page 35, transaction 13
Issue processing transaction: page 35, transaction 14
Issue processing transaction: page 35, transaction 15
Issue processing transaction: page 35, transaction 16
Issue processing transaction: page 35, transaction

...

Processing page 60: 0 transactions found.
Issue processing transaction: page 60, transaction 0
Issue processing transaction: page 60, transaction 1
Issue processing transaction: page 60, transaction 2
Issue processing transaction: page 60, transaction 3
Issue processing transaction: page 60, transaction 4
Issue processing transaction: page 60, transaction 5
Issue processing transaction: page 60, transaction 6
Issue processing transaction: page 60, transaction 7
Issue processing transaction: page 60, transaction 8
Issue processing transaction: page 60, transaction 9
Issue processing transaction: page 60, transaction 10
Issue processing transaction: page 60, transaction 11
Issue processing transaction: page 60, transaction 12
Issue processing transaction: page 60, transaction 13
Issue processing transaction: page 60, transaction 14
Issue processing transaction: page 60, transaction 15
Issue processing transaction: page 60, transaction 16
Issue processing transaction: page 60, transaction

Processing page 61: 0 transactions found.
Issue processing transaction: page 61, transaction 0
Issue processing transaction: page 61, transaction 1
Issue processing transaction: page 61, transaction 2
Issue processing transaction: page 61, transaction 3
Issue processing transaction: page 61, transaction 4
Issue processing transaction: page 61, transaction 5
Issue processing transaction: page 61, transaction 6
Issue processing transaction: page 61, transaction 7
Issue processing transaction: page 61, transaction 8
Issue processing transaction: page 61, transaction 9
Issue processing transaction: page 61, transaction 10
Issue processing transaction: page 61, transaction 11
Issue processing transaction: page 61, transaction 12
Issue processing transaction: page 61, transaction 13
Issue processing transaction: page 61, transaction 14
Issue processing transaction: page 61, transaction 15
Issue processing transaction: page 61, transaction 16
Issue processing transaction: page 61, transaction

Processing page 62: 0 transactions found.
Issue processing transaction: page 62, transaction 0
Issue processing transaction: page 62, transaction 1
Issue processing transaction: page 62, transaction 2
Issue processing transaction: page 62, transaction 3
Issue processing transaction: page 62, transaction 4
Issue processing transaction: page 62, transaction 5
Issue processing transaction: page 62, transaction 6
Issue processing transaction: page 62, transaction 7
Issue processing transaction: page 62, transaction 8
Issue processing transaction: page 62, transaction 9
Issue processing transaction: page 62, transaction 10
Issue processing transaction: page 62, transaction 11
Issue processing transaction: page 62, transaction 12
Issue processing transaction: page 62, transaction 13
Issue processing transaction: page 62, transaction 14
Issue processing transaction: page 62, transaction 15
Issue processing transaction: page 62, transaction 16
Issue processing transaction: page 62, transaction

Processing page 63: 0 transactions found.
Issue processing transaction: page 63, transaction 0
Issue processing transaction: page 63, transaction 1
Issue processing transaction: page 63, transaction 2
Issue processing transaction: page 63, transaction 3
Issue processing transaction: page 63, transaction 4
Issue processing transaction: page 63, transaction 5
Issue processing transaction: page 63, transaction 6
Issue processing transaction: page 63, transaction 7
Issue processing transaction: page 63, transaction 8
Issue processing transaction: page 63, transaction 9
Issue processing transaction: page 63, transaction 10
Issue processing transaction: page 63, transaction 11
Issue processing transaction: page 63, transaction 12
Issue processing transaction: page 63, transaction 13
Issue processing transaction: page 63, transaction 14
Issue processing transaction: page 63, transaction 15
Issue processing transaction: page 63, transaction 16
Issue processing transaction: page 63, transaction

Processing page 64: 0 transactions found.
Issue processing transaction: page 64, transaction 0
Issue processing transaction: page 64, transaction 1
Issue processing transaction: page 64, transaction 2
Issue processing transaction: page 64, transaction 3
Issue processing transaction: page 64, transaction 4
Issue processing transaction: page 64, transaction 5
Issue processing transaction: page 64, transaction 6
Issue processing transaction: page 64, transaction 7
Issue processing transaction: page 64, transaction 8
Issue processing transaction: page 64, transaction 9
Issue processing transaction: page 64, transaction 10
Issue processing transaction: page 64, transaction 11
Issue processing transaction: page 64, transaction 12
Issue processing transaction: page 64, transaction 13
Issue processing transaction: page 64, transaction 14
Issue processing transaction: page 64, transaction 15
Issue processing transaction: page 64, transaction 16
Issue processing transaction: page 64, transaction

In [14]:
# To validate it worked 
darsham_df

,Id,TransactionId,ProductId,UnitPrice,UnitPriceExcTax,CostPrice,TaxGroupId,TaxAmount,Quantity,DiscountAmount,...,Notes,PrintOnOrder,MultipleChoiceProductId,ParentId,IsTaxExempt,MeasurementDetails,Taxes,MultipleChoiceItems,CourseFired,Date
0,1639869614,467711231,1677551,2.5,2.5,0.25,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639869614, 'TaxRateId'...",[],False,2022-10-01T08:30:52.417
0,1639869615,467711231,7243504,2.5,2.5,0.21,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639869615, 'TaxRateId'...",[],False,2022-10-01T08:30:52.417
0,1639869616,467711231,1677386,3.25,2.70833,0.34,None,0.54,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639869616, 'TaxRateId'...","[{'Id': 1639869618, 'TransactionId': 467711231...",False,2022-10-01T08:30:52.417
0,1639869617,467711231,1677386,3.25,2.70833,0.34,None,0.54,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639869617, 'TaxRateId'...",[],False,2022-10-01T08:30:52.417
0,1639870791,467711606,1677543,2.25,2.25,0.21,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639870791, 'TaxRateId'...",[],False,2022-10-01T08:37:07.043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1662750531,473960082,1677390,3.6,3.0,0.34,None,0.6,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662750531, 'TaxRateId'...","[{'Id': 1662750546, 'TransactionId': 473960082...",False,2022-10-31T16:39:09.927
0,1662750534,473960082,1677402,3.25,2.70833,0.38,None,0.54,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662750534, 'TaxRateId'...","[{'Id': 1662750547, 'TransactionId': 473960082...",False,2022-10-31T16:39:09.927
0,1662750537,473960082,18379592,2.75,2.75,0.0,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662750537, 'TaxRateId'...",[],False,2022-10-31T16:39:09.927
0,1662750540,473960082,21918560,4.4,4.4,0.0,None,0.0,1,2.2,...,None,True,None,None,False,None,"[{'TransactionItemId': 1662750540, 'TaxRateId'...",[],False,2022-10-31T16:39:09.927


In [15]:
# To save the raw data to csv 
darsham_df.to_csv(working_directory+'/1_raw_data/dars_oct22.csv',index=False)

---

# Norwich Data Download

In [11]:
# Norwich Download

# New empty dataframe is created
norwich_df = pd.DataFrame()
# To loop through the page numbers 
for i in range(1, 65): 
    api_url = 'https://api.eposnowhq.com/api/v4/Transaction/GetByDate/2022-10-01/2022-11-01/?page='+str(i)
    token = nor_token
    headers = {'Authorization': "Basic {} ".format(token)}
    auth_response = requests.get(api_url, headers=headers)
    print(f'Processing page {i}: {len(auth_response.json())} transactions found.')
    for j in range(200):
        try:
            for transaction_item in auth_response.json()[j]['TransactionItems']:
                tmp = pd.DataFrame.from_dict(transaction_item, orient = 'index').T
                tmp['Date'] = auth_response.json()[j]['DateTime']
                #print(tmp)
                try:
                    norwich_df = pd.concat([norwich_df, tmp])
                except:
                    print(f'Issue processing transaction: page {i}, transaction {j}')
        except:
            print(f'Issue processing transaction: page {i}, transaction {j}')

Processing page 1: 200 transactions found.
Processing page 2: 200 transactions found.
Processing page 3: 200 transactions found.
Processing page 4: 200 transactions found.
Processing page 5: 200 transactions found.
Processing page 6: 200 transactions found.
Processing page 7: 200 transactions found.
Processing page 8: 200 transactions found.
Processing page 9: 200 transactions found.
Processing page 10: 200 transactions found.
Processing page 11: 200 transactions found.
Processing page 12: 200 transactions found.
Processing page 13: 200 transactions found.
Processing page 14: 200 transactions found.
Processing page 15: 200 transactions found.
Processing page 16: 200 transactions found.
Processing page 17: 200 transactions found.
Processing page 18: 200 transactions found.
Processing page 19: 200 transactions found.
Issue processing transaction: page 19, transaction 73
Issue processing transaction: page 19, transaction 105
Processing page 20: 200 transactions found.
Processing page 21: 

Processing page 38: 0 transactions found.
Issue processing transaction: page 38, transaction 0
Issue processing transaction: page 38, transaction 1
Issue processing transaction: page 38, transaction 2
Issue processing transaction: page 38, transaction 3
Issue processing transaction: page 38, transaction 4
Issue processing transaction: page 38, transaction 5
Issue processing transaction: page 38, transaction 6
Issue processing transaction: page 38, transaction 7
Issue processing transaction: page 38, transaction 8
Issue processing transaction: page 38, transaction 9
Issue processing transaction: page 38, transaction 10
Issue processing transaction: page 38, transaction 11
Issue processing transaction: page 38, transaction 12
Issue processing transaction: page 38, transaction 13
Issue processing transaction: page 38, transaction 14
Issue processing transaction: page 38, transaction 15
Issue processing transaction: page 38, transaction 16
Issue processing transaction: page 38, transaction

Processing page 39: 0 transactions found.
Issue processing transaction: page 39, transaction 0
Issue processing transaction: page 39, transaction 1
Issue processing transaction: page 39, transaction 2
Issue processing transaction: page 39, transaction 3
Issue processing transaction: page 39, transaction 4
Issue processing transaction: page 39, transaction 5
Issue processing transaction: page 39, transaction 6
Issue processing transaction: page 39, transaction 7
Issue processing transaction: page 39, transaction 8
Issue processing transaction: page 39, transaction 9
Issue processing transaction: page 39, transaction 10
Issue processing transaction: page 39, transaction 11
Issue processing transaction: page 39, transaction 12
Issue processing transaction: page 39, transaction 13
Issue processing transaction: page 39, transaction 14
Issue processing transaction: page 39, transaction 15
Issue processing transaction: page 39, transaction 16
Issue processing transaction: page 39, transaction

Processing page 40: 0 transactions found.
Issue processing transaction: page 40, transaction 0
Issue processing transaction: page 40, transaction 1
Issue processing transaction: page 40, transaction 2
Issue processing transaction: page 40, transaction 3
Issue processing transaction: page 40, transaction 4
Issue processing transaction: page 40, transaction 5
Issue processing transaction: page 40, transaction 6
Issue processing transaction: page 40, transaction 7
Issue processing transaction: page 40, transaction 8
Issue processing transaction: page 40, transaction 9
Issue processing transaction: page 40, transaction 10
Issue processing transaction: page 40, transaction 11
Issue processing transaction: page 40, transaction 12
Issue processing transaction: page 40, transaction 13
Issue processing transaction: page 40, transaction 14
Issue processing transaction: page 40, transaction 15
Issue processing transaction: page 40, transaction 16
Issue processing transaction: page 40, transaction

Processing page 41: 0 transactions found.
Issue processing transaction: page 41, transaction 0
Issue processing transaction: page 41, transaction 1
Issue processing transaction: page 41, transaction 2
Issue processing transaction: page 41, transaction 3
Issue processing transaction: page 41, transaction 4
Issue processing transaction: page 41, transaction 5
Issue processing transaction: page 41, transaction 6
Issue processing transaction: page 41, transaction 7
Issue processing transaction: page 41, transaction 8
Issue processing transaction: page 41, transaction 9
Issue processing transaction: page 41, transaction 10
Issue processing transaction: page 41, transaction 11
Issue processing transaction: page 41, transaction 12
Issue processing transaction: page 41, transaction 13
Issue processing transaction: page 41, transaction 14
Issue processing transaction: page 41, transaction 15
Issue processing transaction: page 41, transaction 16
Issue processing transaction: page 41, transaction

...

Processing page 60: 0 transactions found.
Issue processing transaction: page 60, transaction 0
Issue processing transaction: page 60, transaction 1
Issue processing transaction: page 60, transaction 2
Issue processing transaction: page 60, transaction 3
Issue processing transaction: page 60, transaction 4
Issue processing transaction: page 60, transaction 5
Issue processing transaction: page 60, transaction 6
Issue processing transaction: page 60, transaction 7
Issue processing transaction: page 60, transaction 8
Issue processing transaction: page 60, transaction 9
Issue processing transaction: page 60, transaction 10
Issue processing transaction: page 60, transaction 11
Issue processing transaction: page 60, transaction 12
Issue processing transaction: page 60, transaction 13
Issue processing transaction: page 60, transaction 14
Issue processing transaction: page 60, transaction 15
Issue processing transaction: page 60, transaction 16
Issue processing transaction: page 60, transaction

Processing page 61: 0 transactions found.
Issue processing transaction: page 61, transaction 0
Issue processing transaction: page 61, transaction 1
Issue processing transaction: page 61, transaction 2
Issue processing transaction: page 61, transaction 3
Issue processing transaction: page 61, transaction 4
Issue processing transaction: page 61, transaction 5
Issue processing transaction: page 61, transaction 6
Issue processing transaction: page 61, transaction 7
Issue processing transaction: page 61, transaction 8
Issue processing transaction: page 61, transaction 9
Issue processing transaction: page 61, transaction 10
Issue processing transaction: page 61, transaction 11
Issue processing transaction: page 61, transaction 12
Issue processing transaction: page 61, transaction 13
Issue processing transaction: page 61, transaction 14
Issue processing transaction: page 61, transaction 15
Issue processing transaction: page 61, transaction 16
Issue processing transaction: page 61, transaction

Processing page 62: 0 transactions found.
Issue processing transaction: page 62, transaction 0
Issue processing transaction: page 62, transaction 1
Issue processing transaction: page 62, transaction 2
Issue processing transaction: page 62, transaction 3
Issue processing transaction: page 62, transaction 4
Issue processing transaction: page 62, transaction 5
Issue processing transaction: page 62, transaction 6
Issue processing transaction: page 62, transaction 7
Issue processing transaction: page 62, transaction 8
Issue processing transaction: page 62, transaction 9
Issue processing transaction: page 62, transaction 10
Issue processing transaction: page 62, transaction 11
Issue processing transaction: page 62, transaction 12
Issue processing transaction: page 62, transaction 13
Issue processing transaction: page 62, transaction 14
Issue processing transaction: page 62, transaction 15
Issue processing transaction: page 62, transaction 16
Issue processing transaction: page 62, transaction

Processing page 63: 0 transactions found.
Issue processing transaction: page 63, transaction 0
Issue processing transaction: page 63, transaction 1
Issue processing transaction: page 63, transaction 2
Issue processing transaction: page 63, transaction 3
Issue processing transaction: page 63, transaction 4
Issue processing transaction: page 63, transaction 5
Issue processing transaction: page 63, transaction 6
Issue processing transaction: page 63, transaction 7
Issue processing transaction: page 63, transaction 8
Issue processing transaction: page 63, transaction 9
Issue processing transaction: page 63, transaction 10
Issue processing transaction: page 63, transaction 11
Issue processing transaction: page 63, transaction 12
Issue processing transaction: page 63, transaction 13
Issue processing transaction: page 63, transaction 14
Issue processing transaction: page 63, transaction 15
Issue processing transaction: page 63, transaction 16
Issue processing transaction: page 63, transaction

Processing page 64: 0 transactions found.
Issue processing transaction: page 64, transaction 0
Issue processing transaction: page 64, transaction 1
Issue processing transaction: page 64, transaction 2
Issue processing transaction: page 64, transaction 3
Issue processing transaction: page 64, transaction 4
Issue processing transaction: page 64, transaction 5
Issue processing transaction: page 64, transaction 6
Issue processing transaction: page 64, transaction 7
Issue processing transaction: page 64, transaction 8
Issue processing transaction: page 64, transaction 9
Issue processing transaction: page 64, transaction 10
Issue processing transaction: page 64, transaction 11
Issue processing transaction: page 64, transaction 12
Issue processing transaction: page 64, transaction 13
Issue processing transaction: page 64, transaction 14
Issue processing transaction: page 64, transaction 15
Issue processing transaction: page 64, transaction 16
Issue processing transaction: page 64, transaction

In [21]:
# To validate it worked
norwich_df

,Id,TransactionId,ProductId,UnitPrice,UnitPriceExcTax,CostPrice,TaxGroupId,TaxAmount,Quantity,DiscountAmount,...,Notes,PrintOnOrder,MultipleChoiceProductId,ParentId,IsTaxExempt,MeasurementDetails,Taxes,MultipleChoiceItems,CourseFired,Date
0,1639859185,467709891,1677449,2.6,2.16667,0.15,None,0.43,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859185, 'TaxRateId'...",[],False,2022-10-01T07:59:57.423
0,1639859374,467709964,1677429,2.8,2.33333,0.36,None,0.47,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859374, 'TaxRateId'...","[{'Id': 1639859376, 'TransactionId': 467709964...",False,2022-10-01T08:02:21.673
0,1639859375,467709964,1677689,2.5,2.5,0.0,None,0.0,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859375, 'TaxRateId'...",[],False,2022-10-01T08:02:21.673
0,1639859405,467709986,1677432,3.0,2.66667,0.34,None,0.33,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859405, 'TaxRateId'...",[],False,2022-10-01T08:02:57.477
0,1639859653,467710060,13405911,3.0,3.0,0.0,None,0.0,2,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1639859653, 'TaxRateId'...",[],False,2022-10-01T08:04:48.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1640157935,467796419,1677660,2.7,2.25,0.58,None,0.45,1,None,...,RHUBARB,True,None,None,False,None,"[{'TransactionItemId': 1640157935, 'TaxRateId'...",[],False,2022-10-01T13:14:54.173
0,1640157937,467796419,1677664,2.0,1.66667,0.45,None,0.33,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1640157937, 'TaxRateId'...",[],False,2022-10-01T13:14:54.173
0,1640159936,467797000,1677432,3.0,2.66667,0.34,None,0.33,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1640159936, 'TaxRateId'...",[],False,2022-10-01T13:16:08.243
0,1640159938,467797000,7580623,3.7,3.08333,2.58,None,0.62,1,None,...,None,True,None,None,False,None,"[{'TransactionItemId': 1640159938, 'TaxRateId'...",[],False,2022-10-01T13:16:08.243


In [17]:
# To save the raw data to csv
norwich_df.to_csv(working_directory+'/1_raw_data/nor_oct22.csv',index=False)

---

# Data and Column Descriptions
Basic EDA will be carried out on one of the dataframes to identify what format the raw data is collected in.


In [32]:
# To drop the two columns that contains lists for each row
darsham_df_eda = darsham_df.drop(columns=['Taxes', 'MultipleChoiceItems']).copy()

In [33]:
bakery.further_eda(darsham_df_eda)

There are 21 columns and 20768 rows 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20768 entries, 0 to 0
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Id                       20768 non-null  object
 1   TransactionId            20768 non-null  object
 2   ProductId                20768 non-null  object
 3   UnitPrice                20768 non-null  object
 4   UnitPriceExcTax          20768 non-null  object
 5   CostPrice                20768 non-null  object
 6   TaxGroupId               0 non-null      object
 7   TaxAmount                20768 non-null  object
 8   Quantity                 20768 non-null  object
 9   DiscountAmount           266 non-null    object
 10  DiscountReasonId         266 non-null    object
 11  DiscountAmountExcTax     266 non-null    object
 12  RefundReasonId           32 non-null     object
 13  Notes                    1133 non-null   object
 14  Prin

None


The number of missing values in each column are:


Id                             0
TransactionId                  0
ProductId                      0
UnitPrice                      0
UnitPriceExcTax                0
CostPrice                      0
TaxGroupId                 20768
TaxAmount                      0
Quantity                       0
DiscountAmount             20502
DiscountReasonId           20502
DiscountAmountExcTax       20502
RefundReasonId             20736
Notes                      19635
PrintOnOrder                   0
MultipleChoiceProductId    20768
ParentId                   20768
IsTaxExempt                    0
MeasurementDetails         20768
CourseFired                    0
Date                           0
dtype: int64


There are 0 duplicated rows of data in the dataset



,Id,TransactionId,ProductId,UnitPrice,UnitPriceExcTax,CostPrice,TaxGroupId,TaxAmount,Quantity,DiscountAmount,...,DiscountAmountExcTax,RefundReasonId,Notes,PrintOnOrder,MultipleChoiceProductId,ParentId,IsTaxExempt,MeasurementDetails,CourseFired,Date
count,20768,20768,20768,20768.0,20768.0,20768.0,0,20768.0,20768,266.0,...,266.0,32,1133,20768,0,0,20768,0,20768,20768
unique,20768,5986,393,80.0,137.0,42.0,0,143.0,14,77.0,...,78.0,3,503,1,0,0,1,0,1,5986
top,1639859763,472377403,1677674,3.6,3.0,0.0,NaN,0.0,1,1.5,...,3.0,19849,Y,True,NaN,NaN,False,NaN,False,2022-10-24T10:29:22.057
freq,1,31,857,1715.0,2783.0,10432.0,NaN,12710.0,18149,22.0,...,29.0,28,73,20768,NaN,NaN,20768,NaN,20768,31


## Observations 
From the above intital EDA, it can be seen that:
- The raw data consists of multiple rows of data (c.20,000 for this API call but over 500,000 for the full raw data) and 23 columns. 
- There are 4 columns that are completely empty, which are likely to be redundant and be removed in the cleaning process. 
- There are a lot of missing values that need to be dealt with appropriately. 
- There appears to be no dupliacted rows of data, but this is on a small sample of the true full dataset so this will be re-assessed in the cleaning process.
- All the datatypes are object, which will have to be amended to the approprate datatype in the cleaning process.

A data dictionary for the raw data can be found below: 

|Column| Description | 
|:--| :- | 
|Id    |   The ID of the row of data from the till system  |                
|TransactionId| The unique transaction ID | 
|ProductId     |  The ID of of the product(s) bought |                
|UnitPrice      |       The unit price of the product(s) bought |         
|UnitPriceExcTax |     The unit price of the product(s) bought excluding tax |           
|CostPrice        |    The cost price of the product(s) that is stored in the system |           
|TaxGroupId        |  The tax group ID of the product(s) |
|Quantity         |    The quantity of the product bought in that transaction |         
|DiscountAmount  |    The discount amount on the transaction |        
|DiscountReasonId |     The ID of the discount reason on the transaction | 
|DiscountAmountExcTax|  The discount amount excluding tax |      
|RefundReasonId     |    The ID of the refund reason |     
|Notes               |   Additional notes added to the order |     
|PrintOnOrder         |    Boolean if the order was printed |       
|MultipleChoiceProductId | Feature of the till system not utilised by the bakery |   
|ParentId       |   Feature of the till system not utilised by the bakery |          
|IsTaxExempt     | Boolean if the order is tax exempt or not |               
|MeasurementDetails |  Feature of the till system not utilised by the bakery |
|Taxes| List of dictionaries containing Tax information | 
|MultipleChoiceItems| List of dictionaries containing the selection of different flavours available of that product | 
|CourseFired         |   Feature of the till system not utilised by the bakery |        
|Date                 | The date of the transaction |           

---

# Next Steps
- The weather data will also be cleaned and explored to ensure it is clean, in the correct and required format for the modelling phase, this can be found here [Weather Cleaning and EDA](./2_Weather_Cleaning_EDA.ipynb). 

- The raw bakery transaction data needs to be explored for missing, duplicated or erronous data and cleaned to remove redundant columns. This is carried out in [Bakery Data Cleaning Notebook](./3_Bakery_Data_Cleaning.ipynb).



>[Return to Contents](#Contents)